### Step 1: Import Datasets and merge

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load macroeconomic and forex datasets
macro = pd.read_csv("macro_data.csv", parse_dates=["DATE"]).set_index("DATE")
forex = pd.read_csv("forex_merged_cleaned.csv", parse_dates=["DATE"]).set_index("DATE")

In [3]:
print("Macro Data:")
print(macro.head(25))

print("\nForex Data:")
print(forex.head(25))

Macro Data:
            Trade Balance  Inflation (CPI)  Core Inflation  Interest Rate  \
DATE                                                                        
2002-01-01       -29678.0            177.7           188.7           1.73   
2002-02-01       -32501.0            178.0           189.1           1.74   
2002-03-01       -31161.0            178.5           189.2           1.73   
2002-04-01       -33965.0            179.3           189.7           1.75   
2002-05-01       -34026.0            179.5           190.0           1.75   
2002-06-01       -34732.0            179.6           190.2           1.75   
2002-07-01       -33913.0            180.0           190.5           1.73   
2002-08-01       -36385.0            180.5           191.1           1.74   
2002-09-01       -36088.0            180.8           191.3           1.75   
2002-10-01       -35095.0            181.2           191.5           1.75   
2002-11-01       -39515.0            181.5           191.9      

In [10]:
# Convert exchange rates to log returns
log_returns = np.log(forex / forex.shift(1))
log_returns = log_returns.dropna()
log_returns.columns = [col + " Return" for col in log_returns.columns]

In [12]:
# Align and merge macro with log returns on DATE
full_df = pd.merge(macro, log_returns, left_index=True, right_index=True, how='inner')
# Drop rows with any missing data (can customize later)
full_df = full_df.dropna()
print(full_df.head(25))

            Trade Balance  Inflation (CPI)  Core Inflation  Interest Rate  \
DATE                                                                        
2002-03-01       -31161.0            178.5           189.2           1.73   
2002-04-01       -33965.0            179.3           189.7           1.75   
2002-05-01       -34026.0            179.5           190.0           1.75   
2002-06-01       -34732.0            179.6           190.2           1.75   
2002-07-01       -33913.0            180.0           190.5           1.73   
2002-08-01       -36385.0            180.5           191.1           1.74   
2002-09-01       -36088.0            180.8           191.3           1.75   
2002-10-01       -35095.0            181.2           191.5           1.75   
2002-11-01       -39515.0            181.5           191.9           1.34   
2002-12-01       -43607.0            181.8           192.1           1.24   
2003-01-01       -41156.0            182.6           192.4           1.24   